## RDC01 de HITES-ACFIN

#### Leer archivo SFTP --> ACFIN

##### Esta Informacion se guardara en las siguientes rutas:
##### -  Archivo EXCEL RDC01 --> Path.home() / "EF Securitizadora" / "Operaciones - Documentos"/ "Documentos"/ "Operaciones"/ "REDEC"/ "HITES_ACFIN" / "ARCHIVOS_XLSX"
##### -  Archivo txt --> Path.home() / "EF Securitizadora" / "Operaciones - Documentos"/ "Documentos"/ "Operaciones"/ "REDEC"/ "HITES_ACFIN" /"ARCHIVOS_TXT"
##### -  Caratula --> Path.home() / "EF Securitizadora" / "Operaciones - Documentos"/ "Documentos"/ "Operaciones"/ "REDEC"/ "HITES_ACFIN" / "CARATULA"

#### Para ejecutar este codigo, solo se debe modificar la variable "fecha_reporte" en formato YYYYMMDD
#### solo se descargara la informacion correspondiente a dicha fecha, si existe otro archivo con otra fecha distinta se ignorara.


In [1]:
import paramiko
from pathlib import Path
import pandas as pd

# ==========================================================
# 📅 FECHA DE REPORTE (YYYYMMDD) → AJUSTA ESTA
# ==========================================================
fecha_reporte = "20251121"

# ==========================================================
# ⚙️ Parámetros de conexión SFTP
# ==========================================================
host = "208.109.33.151"
port = 22
username = "redecacfin"

key_path = (
    Path.home()
    / "EF Securitizadora"
    / "Operaciones - Documentos"
    / "Documentos"
    / "Operaciones"
    / "REDEC"
    / "HITES_ACFIN"
    / "SFTP"
    / "redecacfin"
)

# ==========================================================
# 📂 Directorios locales
# ==========================================================
base_dir = (
    Path.home()
    / "EF Securitizadora"
    / "Operaciones - Documentos"
    / "Documentos"
    / "Operaciones"
    / "REDEC"
    / "HITES_ACFIN"
)

local_temp = base_dir / "SFTP" / "ARCHIVOS"
local_temp.mkdir(parents=True, exist_ok=True)

ruta_caratula = base_dir / "CARATULA"
ruta_caratula.mkdir(parents=True, exist_ok=True)

ruta_txt = base_dir / "archivo_txt"
ruta_txt.mkdir(parents=True, exist_ok=True)

ruta_xlsx = base_dir / "archivo_xlsx"
ruta_xlsx.mkdir(parents=True, exist_ok=True)

print("\n📁 Carpetas verificadas:")
print(f"   - Temporal: {local_temp}")
print(f"   - CARATULA: {ruta_caratula}")
print(f"   - archivo_txt: {ruta_txt}")
print(f"   - archivo_xlsx: {ruta_xlsx}")

# ==========================================================
# 🔐 Conectar al servidor SFTP
# ==========================================================
key = paramiko.RSAKey.from_private_key_file(str(key_path))
transport = paramiko.Transport((host, port))
transport.connect(username=username, pkey=key)
sftp = paramiko.SFTPClient.from_transport(transport)

remote_dir = "/home/redecacfin/EFSecuritizadora"

archivos_remotos = sftp.listdir(remote_dir)

print("\n📂 Archivos disponibles en el servidor:")
for archivo in archivos_remotos:
    print("  -", archivo)

# ==========================================================
# 🔽 DESCARGA FILTRADA POR FECHA + CLASIFICACIÓN
# ==========================================================
print(f"\n🔍 Filtrando archivos con fecha {fecha_reporte}\n")

for archivo in archivos_remotos:

    archivo_up = archivo.upper()

    # ❌ Saltar versiones temporales
    if archivo_up.endswith("V1.TXT"):
        print(f"⏭️ Ignorado (v1 temporal): {archivo}")
        continue

    # ❌ Saltar archivos que no contienen la fecha
    if fecha_reporte not in archivo:
        print(f"⏭️ Ignorado (fecha no coincide): {archivo}")
        continue

    # Descargar archivo
    remote_file = f"{remote_dir}/{archivo}"
    temp_file = local_temp / archivo

    print(f"\n⬇️ Descargando archivo válido: {archivo}")
    sftp.get(remote_file, str(temp_file))
    print("   ✅ Descarga completada.")

    # Clasificación automática
    if archivo_up.startswith("CARATULA_"):
        destino = ruta_caratula / archivo
    elif archivo_up.startswith("RDC01_"):
        destino = ruta_txt / archivo
    else:
        destino = temp_file

    # mover si corresponde
    if destino != temp_file:
        temp_file.rename(destino)
        print(f"   📁 Movido a: {destino}")
    else:
        print("   ℹ️ Archivo sin clasificación, queda en ARCHIVOS/")

# cerrar conexión
sftp.close()
transport.close()

print("\n✅ Descarga y clasificación completadas.\n")

# ==========================================================
# 🔄  CONVERTIR SOLO EL TXT DE LA FECHA A XLSX
# ==========================================================
print("\n🔄 Buscando archivo TXT correspondiente a la fecha del reporte...\n")

# Buscar solo TXT que contengan la fecha exacta
txt_files = [f for f in ruta_txt.glob("*.TXT") if fecha_reporte in f.name]

if not txt_files:
    print(f"❌ No existe archivo TXT para la fecha {fecha_reporte} en {ruta_txt}")
else:
    print(f"📌 Archivo encontrado para la fecha {fecha_reporte}: {txt_files[0].name}")

    file_txt = txt_files[0]  # Solo el archivo correcto

    # Ruta de salida
    salida_xlsx = ruta_xlsx / (file_txt.stem + ".xlsx")

    # Estructura RDC01
    columnas = [
        "RUT", "TIPO_PERSONA", "CODIGO_OPERACION", "OPERACION_TITULO_III",
        "TIPO_DEUDOR", "TIPO_DE_CREDITO", "FECHA_OTORGAMIENTO", "CARGA_FINANCIERA",
        "FECHA_EXTINCION_OPERACION", "VALOR_GARANTIA_INMOBILIARIA", 
        "VALOR_GARANTIA_MOBILIARIA", "VALOR_GARANTIA_FINANCIERA",
        "VALOR_GARANTIA_AVAL_FIANZA", "MONTO_ORIGINAL_OPERACION",
        "MONTO_ACTUAL_OPERACION", "MONTO_AL_DIA", 
        "MONTO_MORA_1_TRAMO", "MONTO_MORA_2_TRAMO", "MONTO_MORA_3_TRAMO",
        "MONTO_MORA_4_TRAMO", "MONTO_MORA_5_TRAMO", "MONTO_MORA_6_TRAMO",
        "MONTO_MORA_7_TRAMO", "MONTO_MORA_8_TRAMO", "MONTO_MORA_9_TRAMO",
        "MORA_ACTUAL", "DEUDA_RENEGOCIADA", "DEUDA_ACELERADA"
    ]

    anchos = [
        10, 1, 30, 1, 1, 2, 8, 15, 8,
        15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        4, 1, 1
    ]

    columnas_float = [
        'CARGA_FINANCIERA', 'VALOR_GARANTIA_INMOBILIARIA', 'VALOR_GARANTIA_MOBILIARIA',
        'VALOR_GARANTIA_FINANCIERA', 'VALOR_GARANTIA_AVAL_FIANZA',
        'MONTO_ORIGINAL_OPERACION', 'MONTO_ACTUAL_OPERACION', 'MONTO_AL_DIA',
        'MONTO_MORA_1_TRAMO', 'MONTO_MORA_2_TRAMO', 'MONTO_MORA_3_TRAMO',
        'MONTO_MORA_4_TRAMO', 'MONTO_MORA_5_TRAMO', 'MONTO_MORA_6_TRAMO',
        'MONTO_MORA_7_TRAMO', 'MONTO_MORA_8_TRAMO', 'MONTO_MORA_9_TRAMO',
        'MORA_ACTUAL'
    ]

    columnas_fecha = ["FECHA_OTORGAMIENTO", "FECHA_EXTINCION_OPERACION"]

    # Leer archivo de ancho fijo
    df = pd.read_fwf(
        file_txt,
        widths=anchos,
        names=columnas,
        dtype=str,
        skiprows=1
    )

    # Limpiar espacios
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Convertir números
    for col in columnas_float:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Convertir fechas
    for col in columnas_fecha:
        df[col] = pd.to_datetime(df[col], format="%Y%m%d", errors="coerce")

    # Exportar a Excel
    df.to_excel(salida_xlsx, index=False)

    print(f"   ✅ Convertido a XLSX → {salida_xlsx.name}")

print("\n🎉 Conversión completada SOLO para el archivo de la fecha requerida.\n")


📁 Carpetas verificadas:
   - Temporal: C:\Users\MarceloVidal\EF Securitizadora\Operaciones - Documentos\Documentos\Operaciones\REDEC\HITES_ACFIN\SFTP\ARCHIVOS
   - CARATULA: C:\Users\MarceloVidal\EF Securitizadora\Operaciones - Documentos\Documentos\Operaciones\REDEC\HITES_ACFIN\CARATULA
   - archivo_txt: C:\Users\MarceloVidal\EF Securitizadora\Operaciones - Documentos\Documentos\Operaciones\REDEC\HITES_ACFIN\archivo_txt
   - archivo_xlsx: C:\Users\MarceloVidal\EF Securitizadora\Operaciones - Documentos\Documentos\Operaciones\REDEC\HITES_ACFIN\archivo_xlsx

📂 Archivos disponibles en el servidor:
  - CARATULA_RDC01_20251107v1.TXT
  - RDC01_1234567890_20251107v1.TXT
  - CARATULA_RDC01_20251107.TXT
  - RDC01_1234567890_20251107.TXT
  - CARATULA_RDC01_20251114.TXT
  - RDC01_1234567890_20251114.TXT

🔍 Filtrando archivos con fecha 20251121

⏭️ Ignorado (v1 temporal): CARATULA_RDC01_20251107v1.TXT
⏭️ Ignorado (v1 temporal): RDC01_1234567890_20251107v1.TXT
⏭️ Ignorado (fecha no coincide): CAR